In [1]:
!pip install ultralytics
!pip install easyocr

In [2]:
from ultralytics import YOLO
import cv2
from sort.sort import Sort
import numpy as np


import string
import easyocr


In [3]:
def fetch_car_associated_with_plates(license_plate, vehicle_track_ids):

    x1, y1, x2, y2, confidenceScore, class_id = license_plate

    foundCarBoxContainingLicensePlate = False
    car_index = -1  # Initialize with -1 to handle cases where no car is found
    
    # Loop through each car's bounding box to find if it contains the license plate
    for i, (x1car, y1car, x2car, y2car, car_id) in enumerate(vehicle_track_ids):
        if x1 > x1car and y1 > y1car and x2 < x2car and y2 < y2car:
            car_index = i
            foundCarBoxContainingLicensePlate = True
            break

    if foundCarBoxContainingLicensePlate:
        return vehicle_track_ids[car_index]

    # Return a tuple of -1s if no car contains the license plate
    return -1, -1, -1, -1, -1


In [4]:

# Dictionaries to handle special chars!
dictionary_for_special_char_to_int = {'A': '4', 'G': '6', 'I': '1', 'J': '3', 'O': '0', 'S': '5'}
dictionary_for_special_int_to_char = {'0': 'O', '1': 'I', '3': 'J', '4': 'A', '5': 'S', '6': 'G' }


def check_license_compliability_with_state_format(text):
    if len(text) != 7:
        return False

    # Check first three characters
    if not all(char in string.ascii_uppercase or char in dictionary_for_special_int_to_char.keys() for char in text[:3]):
        return False

    # Check last four characters for digits
    if not all(char.isdigit() or char in dictionary_for_special_char_to_int.keys() for char in text[3:]):
        return False

    return True


def format_license_according_to_state(text):
    
    license_plate_ = ''
    '''
    If First 3 characters of license plates are int, needs to be converted to chars, &
    if Last 4 characters of license plates are chars, needs to be converted to int
    '''
    mapping = {0: dictionary_for_special_int_to_char, 
               1: dictionary_for_special_int_to_char,
               2: dictionary_for_special_int_to_char,
               3: dictionary_for_special_char_to_int, 
               4: dictionary_for_special_char_to_int, 
               5: dictionary_for_special_char_to_int, 
               6: dictionary_for_special_char_to_int}
    
    formatted_license_plate_number = ''.join(mapping[j].get(text[j], text[j]) for j in range(7))

    return formatted_license_plate_number

In [5]:
reader = easyocr.Reader(['en'], gpu=False)

def read_and_process_license_plate(license_plate_crop):
    detections = reader.readtext(license_plate_crop)

    for boundingBox, text, confidenceScore in detections:
        text = text.upper().replace(' ', '')
        # print("COMPLIES ???????? ", check_license_compliability_with_state_format(text))

        if check_license_compliability_with_state_format(text):
            formatted_license, confidenceScore = format_license_according_to_state(text), confidenceScore
            return formatted_license, confidenceScore

    return None, None

Using CPU. Note: This module is much faster with a GPU.


In [6]:


# For keeping track of all objs in vid
vehicle_tracer = Sort()

yolo_coco_model = YOLO('yolov8n.pt') 
license_plate_detector = YOLO('./models/license_plate_detector.pt') # pre-trained model

capture = cv2.VideoCapture('./carVideo.mp4')

vehicles = [2,3,5,7]

store_final_info = {}
frameNo = -1
isReturningFrame = True
while isReturningFrame: 

    isReturningFrame, frame = capture.read()
    frameNo += 1

    if not isReturningFrame:
        break

    store_final_info[frameNo] = {}
    
    detections = yolo_coco_model(frame)[0]

    detections_ = [[x1, y1, x2, y2, confidence_score]
                   for x1, y1, x2, y2, confidence_score, class_id in detections.boxes.data.tolist()
                   if int(class_id) in vehicles]


    if not detections_:
        print("~~~~~~~~~~~~~~~ SKIPPED A FRAME DUE TO NO DETECTIONS !!!!!! ~~~~~~~~~~~~~~~~~")
        continue  # Skip this frame if no vehicle detections

    # Ensure detections_ is correctly shaped as a NumPy array
    detections_ = np.array(detections_)
    if detections_.ndim != 2 or detections_.shape[1] != 5:
        print("################# SKIPPED A FRAME DUE TO INCORRECT SHAPE !!!!!! #################")
        continue  # Skip this frame if detection shapes are incorrect
    

    # Tracking the vehicles
    # track_ids = vehicle_tracer.update(np.asarray(detections_))
    track_ids = vehicle_tracer.update(detections_)

    # Detecting the license plates
    license_plates = license_plate_detector(frame)[0]
    for license_plate in license_plates.boxes.data.tolist():
        x1, y1, x2, y2, confidence_score, class_id = license_plate

        # Finding the car associated with the license plate
        x1car, y1car, x2car, y2car, carId = fetch_car_associated_with_plates(license_plate, track_ids)

        if carId != -1: 
        
            # Cropping our license plate
            cropped_license_plate = frame[int(y1):int(y2) , int(x1):int(x2) , :]

            # Processing our license plate
            cropped_license_plate_gray = cv2.cvtColor(cropped_license_plate, cv2.COLOR_BGR2GRAY)
            _, cropped_license_plate_thresholded = cv2.threshold(cropped_license_plate_gray, 64, 255, cv2.THRESH_BINARY_INV)

            # cv2.imshow("original_crop",cropped_license_plate)
            # cv2.imshow("thresholded_crop",cropped_license_plate_thresholded)
            # cv2.waitKey(0)

            # Read license plate number
            license_text, license_text_confidence_score = read_and_process_license_plate(cropped_license_plate_thresholded)
            # print("license_text---->",license_text)
            # print("license_text_confidence_score---->",license_text_confidence_score)
            if license_text : 
                store_final_info[frameNo][carId] = {
                    'car' : {'carBoundingBox': [x1car, y1car, x2car, y2car]}, 
                    'license_plate': {
                        'licenseBoundingBox': [x1, y1, x2, y2],
                        'licenseText': license_text,
                        'licenseBoundingBoxScore': confidence_score,
                        'licenseTextConfidenceScore': license_text_confidence_score
                    }
                }


# print("store_final_info--------->", store_final_info)     




0: 384x640 3 cars, 318.3ms
Speed: 7.0ms preprocess, 318.3ms inference, 170.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 60.2ms
Speed: 4.0ms preprocess, 60.2ms inference, 13.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 16.4ms
Speed: 6.0ms preprocess, 16.4ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 17.7ms
Speed: 6.0ms preprocess, 17.7ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 13.6ms
Speed: 4.9ms preprocess, 13.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 15.5ms
Speed: 1.9ms preprocess, 15.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 13.5ms
Speed: 5.0ms preprocess, 13.5ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 44.0ms
Speed: 6.8ms preprocess, 44.0ms inference, 5.0ms postprocess per

In [7]:
def write_final_info_to_csv(results, output_path):

    with open(output_path, 'w') as file:
        header = '{},{},{},{},{},{},{}\n'.format('frameNo', 'carId', 'carBoundingBox', 'licenseBoundingBox', 'licenseBoundingBoxScore', 'licenseText', 'licenseTextConfidenceScore')

        file.write(header)
        
        # print("results-----------> ",results)
        # print("results.keys()-----------> ",results.keys())
        for frameNo in results.keys():
            for carId in results[frameNo].keys():
                carInfo = results[frameNo][carId]
                # print("carInfo--------------------> ",carInfo)
                if 'car' in carInfo and 'license_plate' in carInfo and 'licenseText' in carInfo['license_plate']:
                    
                    file.write('{},{},{},{},{},{},{}\n'.format(
                        frameNo,
                        carId,
                        '[{} {} {} {}]'.format(
                            carInfo['car']['carBoundingBox'][0], 
                            carInfo['car']['carBoundingBox'][1], 
                            carInfo['car']['carBoundingBox'][2],
                            carInfo['car']['carBoundingBox'][3]),
                        '[{} {} {} {}]'.format(
                            carInfo['license_plate']['licenseBoundingBox'][0],
                            carInfo['license_plate']['licenseBoundingBox'][1],
                            carInfo['license_plate']['licenseBoundingBox'][2],
                            carInfo['license_plate']['licenseBoundingBox'][3]),
                        carInfo['license_plate']['licenseBoundingBoxScore'],
                        carInfo['license_plate']['licenseText'],
                        carInfo['license_plate']['licenseTextConfidenceScore']))
                else:
                    print("SKIPPED WRITING TO THE FILE")
        file.close()

In [8]:

# Writing final resultant info to a CSV file.
write_final_info_to_csv(store_final_info, './data_records.csv')